In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
from keplergl import KeplerGl
#import pprint

In [2]:

case_type = "Recovered"
data_dir = os.path.join("..", "COVID-19", "csse_covid_19_data", "csse_covid_19_time_series")
df = pd.read_csv(os.path.join(data_dir, "time_series_covid19_{}_global.csv".format(case_type.lower())))

In [3]:
info_col = ["Province/State", "Country/Region", "Lat", "Long"]

col_dayend = []
col_dayend_str = []
for col_str in df.columns:
    if not(col_str in info_col):
        col_obj = datetime.strptime(col_str, "%m/%d/%y") # parse 1/21/2020 10:00 PM to the format
        #print(col_obj.date())
        #print(col_obj)
        if col_obj.date() in col_dayend:
            col_dayend = col_dayend[:-1]
            col_dayend_str = col_dayend_str[:-1]
        col_dayend.append(col_obj.date())
        col_dayend_str.append(col_str)
        #print(col)
        #print(type(col))
#len(col_dayend)
#print(col_dayend_str)

In [4]:
info_col2 = [ c for c in info_col if not c=="First confirmed date in country (Est.)" ]
df2 = df[info_col2+col_dayend_str]
df3 = pd.melt(df2, id_vars=info_col2, value_vars=col_dayend_str, var_name='datetime', value_name='confirmed_case')
df3 = df3.dropna(subset=["confirmed_case"])
# this is to show the data using the time slider
dates = []
for d in df3.datetime.tolist():
    dt_obj = datetime.strptime(d, "%m/%d/%y") # parse 1/21/2020 10:00 PM to the format
    dates.append(dt_obj.strftime("%Y-%m-%d")+" 00:00")
df3['datetime'] = dates

In [5]:
point_gdf = gpd.GeoDataFrame(df3, geometry=gpd.points_from_xy(df3["Long"], df3["Lat"]))

In [6]:
%run map_time_series-recovered.conf.py
# use the previous config as base
#config

In [7]:
map_1 = KeplerGl(height=900, config=config)
map_1.add_data(data=point_gdf, name="confirmed_case")

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': 'confirmed_case', 'name': 'da…

In [8]:
# add the time slider to map, need to manually add this to the config, because the save_to_html won't recognize the filter part

map_1.config["config"]['visState']["filters"] = [{"dataId": 'confirmed_case', "name": "datetime", 
                                                      'type': 'timeRange',
                                                      'value': [1579651200000, 1579737600000], ## a day
                                                      'name': 'datetime'}]


In [9]:
map_1.save_to_html(file_name="map_covid19_{}.html".format(case_type), config=map_1.config, read_only=True)

Map saved to map_covid19_Recovered.html!
